# 第十一章 死锁

## 11.1 死锁问题

一组阻塞的进程持有一种资源等待获取另一个进程所占有的一个资源。

## 11.2 系统模型

**资源**：处理器，I/O 通道，主和副存储器，设备和数据结构。
* 在一个时间只能一个进程使用且不能被删除。
* 进程获得资源，后来释放由其他进程重用。

**资源分配图**：用于表示资源的使用情况，一组定点V和边E的集合。
* V有两种类型：P（系统中的所有进程）和R（系统中所有资源类型）。
* requesting/claiming edge - directed edge $P_i \rightarrow R_j$。
* assignment/holding edge - directed edge $R_j \rightarrow P_i$。

如果资源分配图为一个有向图，如果图中存在环的话，则说明有可能产生死锁。即有死锁的话，资源分配图中一定有环；但是有环的话，不一定存在死锁，还和资源的数量有关。

## 11.3 死锁的特征

死锁可能出现如果四个条件同时成立：
* 互斥：在一个时间只能有一个进程使用资源。
* 持有并等待：进程保持至少一个资源正在等待获取其他进程持有的额外资源。
* 无抢占：一个资源只能被进程自愿释放。
* 循环等待：在等待进程几何（$P_0, P_1, \dots, P_N$)，$P_0$正等待$P_1$所占资源，$P_1$正在等待$P_2$占用的资源，$\dots$，$P_N$正在等待$P_0$所占用的资源。

## 11.4 死锁预防

**死锁预防（Deadlock Prevention）的基本思路**
* 确保系统永远不会进入死锁状态。
* 运行系统进入死锁状态，然后恢复。
* 忽略这个问题，假装系统中从来没有发生死锁。

死锁预防主要是会限制申请方式，**主要的方法**是破坏死锁产生的必要条件：
* 互斥：将共享资源从互斥转变为不互斥的资源。
    * 但是一般资源不具备这种特征，可能会引入程序不确定性的错误。
* 占用并等待：必须保证当一个进程请求的资源，它不持有任何其他资源。
    * 但是这种方法资源利用率低，可能发生饥饿。
* 无抢占：如果进程占有某些资源并请求它不能被立即分配的资源，则释放当前正占有的资源。
    * 系统中想要占用资源的进程释放资源，只能强制杀死该进程。
* 循环等待：对所有资源类型进行排序，并要求每个进程按照资源的顺序进行申请。
    * 传统的操作系统中用的比较少，但是嵌入式系统中用的比较多，因为在嵌入式系统中资源的类型有限。

## 11.5 死锁避免

**死锁避免（Deadlock Avoidance）**：在进程申请资源时，系统判断分配资源后是否会产生死锁，如果有可能产生死锁，则不进行分配。
* 每个进程需要预先声明使用每个类型资源的最大数量。
* 资源的分配状态是通过限定提供与分配的资源数量和进程的最大需求。
* 动态检查资源分配状态，以确保永远不会有一个环形等待状态。

**安全状态**：针对系统中所有的序列，存在安全序列，这个序列中每个进程要求的资源能够由当前可用的资源+所有前面进程持有的资源来满足。
* 状态包含安全状态和不安全状态，不安全状态包含但不等于死锁状态。

### 11.5.1 银行家算法

**前提条件**：
* 多个实例。
* 每个进程都必须能最大限度地利用资源
* 当一个进程请求一个资源，就不得不等待。
* 当一个进程获得所有的资源就必须在一段有限的时间释放他们。

基于这些前提条件，银行家算法通过尝试寻找一个进程执行序列，来决定一个状态是否是安全的。

**数据结构**
* n：进程数量。
* m：资源类型数量。
* Max：总需求量，$n \times m$阶的矩阵。
* Available：剩余空闲量，长度为m的向量，表示资源的剩余空闲量。
* Allocation：已分配量，$n \times m$阶的矩阵。
* Need：未来需要量，$n \times m$阶的矩阵。
* $Need[i][j] = Max[i][j] - Allocation[i][j]$

**安全状态检查**：
1. 首先进行初始化。
    * $work = available$
    * $Finish[i] = false$，当前系统中未结束的进程
2. 找到一个进程i，满足以下条件，如果没有找到，则跳转到第4步
    * $Finish[i] = false$，一个未结束的进程
    * $Need_i <= work$
3. 表明进程i满足正常执行的条件，可以释放它的资源然后继续判断
    * $work = work + Allocation_i$
    * $Finish[i] = ture$
4. 如果所有进程都可以结束，则说明是安全的，否则是不安全的。

**银行家算法**

假设一个进程P现在申请Request（一个向量，表示每个类型的资源申请数量）资源。
1. 如果$Request > Need_i$，则提出错误，因为进程申请的数量大于预先声明的资源需求数量。
2. 如果$Request > Available$，则进程需要等待，因为资源不可用。
3. 通过修改状态来分配资源给进程P
    * $Available = Available - Request$
    * $Need_i = Need_i - Request$
    * $Allocation_i = Allocation_i + Request$
4. 调用安全检查判断分配资源给进程P后，系统是否处于安全状态。

## 11.6 死锁检测

**死锁检测（Deadlock Detection）**：系统定期检查是否有死锁。
* 允许系统进入死锁状态。
* 死锁检测算法。
* 恢复机制。

死锁检测算法类似银行家算法中的安全状态检测，由于死锁检测的复杂度为$O(m*n^2)$，并且一般需要预知进程所需资源的数量。所以死锁检测一般用于调试，而不是在实际的系统中。

## 11.7 死锁恢复

**死锁恢复（Recovery from Deadlock）**：发现死锁后，通过某些方式来破坏死锁。
* 终止所有的死锁进程。
* 在一个时间内终止一个进程直到死锁解除。
* 回滚，返回一些安全状态，重启进程到安全状态。

## 11.8 IPC 概述

进程通信（Inter Process Communication）分为直接通信和间接通信：
* 直接通信：两个进程通过内核传递消息，消息的接收方和发送方必须明确（Sender -> Kernel -> Receiver）
* 间接通信：内核为需要通信的进程开辟一个共享空间，发送方将数据放入共享空间，接收方从共享空间读取数据。

通信分为阻塞和非阻塞：
* 阻塞表示发送方需要等待接收方完整的收到消息。
* 非阻塞表示发送方不会等待接收方完整的接收消息。

**通信链路缓冲**
* 缓冲区为0，对应为阻塞式通信。
* 缓冲区有限，当缓冲区满的情况下，发送方需要等待。
* 缓冲区无限，发送方无需等待。

## 11.9 进程通信的方式

### 11.9.1 信号（Signal）

通过软件中断通知数据处理，例如：SIGFPE、SIGKILL、SIGSTOP。

接收到信号的**处理方式**：
* Catch：指定信号处理函数被调用。
* Ignore：依靠操作系统的默认操作。
* Mask：闭塞信号。

**优点**：效率高。

**缺点**：不能传输要交换的任何数据，因为信号量大小基本为1个字节。

### 11.9.2 管道（Pipe）

* 子进程从父进程继承文件描述符。
* 进程不知道自己的文件描述符另一端的设备。

### 11.9.3 消息队列

消息队列按FIFO来管理：
* Message：作为一个字节序列储存。
* Message Queue：消息数组。

### 11.9.4 共享内存

每个进程都有私有的地址空间，在每个地址空间内，明确地设置了共享内存段。

**优点**：快速，方便地共享数据。

**缺点**：必须同步数据的访问。